<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Vantage Analytic Library Demo 

</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Hypothesis Tests</b></p>

<p style = 'font-size:16px;font-family:Arial'>Statistical tests provide a means of testing whether the outcome of an experiment could have been accidental.  Numerous tests are provided in-database, all of the hypothesis testing variety.  These include the following 18 tests, categorized below, that are currently available in the Vantage Analytic Library.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Parametric Tests</b></p>

<p style = 'font-size:16px;font-family:Arial'>Two Sample T-Test for Equal Means</p>
    <li>T Paired</li>
    <li>T Unpaired</li>
    <li>T Unpaired with Indicator</li>
<p style = 'font-size:16px;font-family:Arial'>F-Test – N-Way</p>
<p style = 'font-size:16px;font-family:Arial'>F-Test/Analysis of Variance (Two Way Unequal Sample Size)</p>

<p style = 'font-size:16px;font-family:Arial'><b>Binomial Tests</b>

<li>Binomial/Ztest</li>
<li>Binomial Sign Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'><b>Tests based on Contingency</b>

<li>Chi Square Test</li>
<li>Median Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'><b>Kolmogorov/Smirnoff Tests</b>
    
<li>Kolmogorov/Smirnoff Test (One Sample)</li>
<li>Lilliefors Test</li>
<li>Shapiro-Wilk Test</li>
<li>D’Agostino and Pearson Test</li>
<li>Smirnov Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'><b>Rank Tests</b>

<li>Mann-Whitney/Kruskal-Wallis Test</li>
<li>Mann-Whitney/Kruskal-Wallis Independent Tests</li>
<li>Wilcoxon Signed Ranks Test</li>
<li>Frieman Test with Kendall’s Coefficient of Concordance & Spearman’s Rho</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'>In order to execute the Statistical Tests described in this book, the Statistical Test Metadata tables must be loaded into a database on the system to be analyzed.  These tables have been loaded into the Demo_Financial database along with the other VAL XSP and Table Operators.  The Statistical Test functions all provide a parameter called statsdatabase that can be used to specify the database in which these tables are installed.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Vantage Analytic Library - Call Structure</b></p>

<p style = 'font-size:16px;font-family:Arial'>call val.td_analyze('\<function name\>','\<database=database name\>;\<tablename=table name\>;\<columns=column1, column2\>,'\<param1\>=\<value1\>;\<param2\>=\<value2\>...');</p>

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>1. Start by connecting to the Vantage system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [ ]:
Set query_band='DEMO=VAL-Hypothesis_Tests.ipynb;' update for session;

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>2. Getting Data for This Demo:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>


In [ ]:
call get_data('DEMO_Financial_cloud'); ---takes about 35 seconds
---call get_data('DEMO_Financial_local'); ---takes about 100 seconds

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
call space_report();  -- optional, takes about 10 seconds

<p style = 'font-size:16px;font-family:Arial'>Change focus to the database DEMO_Financial.</p>

In [ ]:
DATABASE DEMO_Financial;

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Demo data - Financial Customers/Accounts/Transactions</b></p>

<p style = 'font-size:16px;font-family:Arial'>The following data has been put into the DEMO_Financial database for the examples in the three different Jupyter Notebooks.  It's a simplistic fictitious dataset of banking customers (approx. 10K rows), Accounts (approx. 20K rows) and Transactions (approx. 1Million rows).  They are related to each other in the following ways:</p>

<img src="images/DemoData.png">

In [ ]:
SELECT * FROM DEMO_Financial.Customer SAMPLE 10;

<p style = 'font-size:16px;font-family:Arial'>Below command gives the metadata for the table.</p>

In [ ]:
%meta

In [ ]:
SELECT * FROM DEMO_Financial.Accounts SAMPLE 10;

In [ ]:
SELECT * FROM DEMO_Financial.Transactions SAMPLE 10;

---

<p style = 'font-size:16px;font-family:Arial'>The following Analytic Data Set (ADS) was created by joining all three tables above:</p>

In [ ]:
CREATE TABLE demo_user.VAL_ADS AS (
    SELECT 
        T1.cust_id  AS cust_id
       ,MIN(T1.income) AS tot_income
       ,MIN(T1.age) AS tot_age
       ,MIN(T1.years_with_bank) AS tot_cust_years
       ,MIN(T1.nbr_children) AS tot_children
       ,CASE WHEN MIN(T1.marital_status) = 1 THEN 1 ELSE 0 END AS single_ind
       ,CASE WHEN MIN(T1.gender) = 'F' THEN 1 ELSE 0 END AS female_ind
       ,CASE WHEN MIN(T1.marital_status) = 2 THEN 1 ELSE 0 END AS married_ind
       ,CASE WHEN MIN(T1.marital_status) = 3 THEN 1 ELSE 0 END AS separated_ind
       ,MAX(CASE WHEN T1.state_code = 'CA' THEN 1 ELSE 0 END) AS ca_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'NY' THEN 1 ELSE 0 END) AS ny_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'TX' THEN 1 ELSE 0 END) AS tx_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'IL' THEN 1 ELSE 0 END) AS il_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'AZ' THEN 1 ELSE 0 END) AS az_resident_ind
       ,MAX(CASE WHEN T1.state_code = 'OH' THEN 1 ELSE 0 END) AS oh_resident_ind
       ,MAX(CASE WHEN T2.acct_type = 'CK' THEN 1 ELSE 0 END) AS ck_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'SV' THEN 1 ELSE 0 END) AS sv_acct_ind
       ,MAX(CASE WHEN T2.acct_type = 'CC' THEN 1 ELSE 0 END) AS cc_acct_ind
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS ck_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS sv_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS cc_avg_bal
       ,AVG(CASE WHEN T2.acct_type = 'CK' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS ck_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'SV' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS sv_avg_tran_amt
       ,AVG(CASE WHEN T2.acct_type = 'CC' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS cc_avg_tran_amt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 1 THEN T3.tran_id ELSE NULL END) AS q1_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 2 THEN T3.tran_id ELSE NULL END) AS q2_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 3 THEN T3.tran_id ELSE NULL END) AS q3_trans_cnt
       ,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 4 THEN T3.tran_id ELSE NULL END) AS q4_trans_cnt
    FROM DEMO_Financial.Customer AS T1
        LEFT OUTER JOIN DEMO_Financial.Accounts AS T2
            ON T1.cust_id = T2.cust_id
        LEFT OUTER JOIN DEMO_Financial.Transactions AS T3
            ON T2.acct_nbr = T3.acct_nbr
GROUP BY T1.cust_id) WITH DATA UNIQUE PRIMARY INDEX (cust_id);

In [ ]:
SELECT * FROM demo_user.VAL_ADS SAMPLE 10;

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. Statistical Tests</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Binomial Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>
<p style = 'font-size:16px;font-family:Arial'>In a binomial test there are assumed to be an independent trials, each with two possible outcomes, each of equal probability.  You may choose to perform a binomial test, in which the sign of the difference between a first and second column is analyzed, or a sign test, in which the sign of a single column is analyzed.  In a binomial test, you may choose to use a probability different from the 0.5 default value, whereas in a sign test, the binomial probability is fixed at 0.5.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **binomialtest**</b></p>


<p style = 'font-size:16px;font-family:Arial'>The binomialtest parameter:</p>
<p style = 'font-size:16px;font-family:Arial'>
    <li>Is required</li>
    <li>Must be the first parameter</li>
    <li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The column to analyze. It must reside in the table indicated by the tablename parameter</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **binomialprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the binomial probability or allow it to default to 0.5.  This parameter may not be used for a sign test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **exactmatches**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This parameter determines what category to place exact matches in.  If zero, exact match is discarded.  If positive, match is placed with values greater than or equal to zero.  If negative or parameter not specified (default), match is placed with values less than or equal to zero.  Note that the exactmatches parameter is not allowed if this is a sign test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>A second column is required when performing a binomial test, and not allowed when performing a sign test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **singletail**</b></p>

<p style = 'font-size:16px;font-family:Arial'>A single-tailed test may be requested by setting this parameter to true.  A two-tailed test may be requested by setting this parameter to false or by not specifying it at all (the default value).  If the binomial probability is not 0.5, singletail must be set to true.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are binomial and sign, representing a binomial or sign test.  If not specified, the default test style is binomial.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected</p>

------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Binomial test producing an unpersisted result set with default values for thresholdprobabilty (.05), binomialprobability (0.5), singletail (false) and exactmatches (negative).</p>

In [ ]:
call val.td_analyze('BinomialTest',
                         'database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          secondcolumn=ck_avg_bal;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI(Unique Primary Index). Otherwise, N is the UPI.

teststyle = binomial (Default))</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>N</td>
    <td>Float</td>  
    <td>Count of value pairs.</td>
  </tr>
  <tr>
    <td>NPos</td>
    <td>Float</td>  
    <td>Count of positive value differences.</td>
  </tr>
  <tr>
    <td>NNeg</td>
    <td>Float</td>  
    <td>Count of negative value differences.</td>
  </tr>
  <tr>
    <td>BO</td>
    <td>Float</td>  
    <td>Binomial probability.</td>
  </tr>
  <tr>
    <td>BinomialCallP</td>
    <td>Char</td>  
    <td>Binomial result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis (positive)</li>
        <li>n=reject null hypothesis (negative)</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>2.  Binomial test producing a result table and introducing a groupby variable gender with default values for other parameters.</p>

In [ ]:
call val.td_analyze('BinomialTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          secondcolumn=ck_avg_bal;
                          groupby=female_ind;
                          outputdatabase=demo_user;
                          outputtablename=_val_binomial_example2;
                          statsdatabase=val;');

In [ ]:
SELECT * FROM demo_user._val_binomial_example2;

<p style = 'font-size:16px;font-family:Arial'>The above output has columns similar to those mentioned in the previous call. We have 2 rows as they are classified based on gender.</p>

<p style = 'font-size:16px;font-family:Arial'>3.  Sign test producing an unpersisted result set with default values for thresholdprobabilty (.05) and singletail (false).</p>

In [ ]:
call val.td_analyze('BinomialTest',
                         'database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          teststyle=sign;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has columns similar to those mentioned in the previous call. We have only 1 row as output as we do not have the group by gender.</p>

<p style = 'font-size:16px;font-family:Arial'>4.  Sign test producing a result table and introducing a groupby variable gender with default values for other parameters.</p>

In [ ]:
call val.td_analyze('BinomialTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumn=sv_avg_bal;
                          groupby=female_ind;
                          outputdatabase=demo_user;
                          outputtablename=_val_binomial_example4;
                          teststyle=sign;
                          statsdatabase=val;');

In [ ]:
SELECT * FROM demo_user._val_binomial_example4;

<p style = 'font-size:16px;font-family:Arial'>The above output has columns similar to those mentioned in the previous call. We have 2 rows as they are classified based on gender.</p>

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Kolmogorov-Smirnoff Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Kolmogorov-Smirnov Goodness-of-Fit Test is used to decide if a sample comes from a population with a specific distribution.  An attractive feature of this test is that the distribution of the K-S test statistic itself does not depend on the underlying cumulative distribution function being tested. Another advantage is that it is an exact test.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The numeric column that is tested to have a normal distribution. It must reside in the table indicated by the tablename parameter</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **kstest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The kstest parameter:
    <li>Is required</li>
    <li>Must be the first parameter</li>
    <li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Smirnov test, this parameter represents a categorical variable with two values that indicate the distribution to which the columnofinterest belongs. It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This parameter is not allowed when using the Smirnov test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>
    
<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are ks, l, sw, p and s, representing a Kolmogorov-Smirnov, Lilliefors, Shapiro-Wilk, D’Agostino and Pearson or Smirnov test.  If not specified, the default test style is Kolmogorov-Smirnov (ks).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected.</p>

--------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Kolmogorov-Smirnov test with group-by option.</p>

In [ ]:
call val.td_analyze('KSTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=KS;
                          groupby=tot_cust_years;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, N is the UPI.

teststyle = ks (Default)</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Klm</td>
    <td>Float</td>  
    <td>Kolmogorov-Smirnov statistic.</td>
  </tr>
  <tr>
    <td>M</td>
    <td>Float</td>  
    <td>Count.</td>
  </tr>
  <tr>
    <td>KlmPValue</td>
    <td>Float</td>  
    <td>Probability associated with Kolmogorov-Smirnov statistic.</td>
  </tr>
  <tr>
    <td>KlmPText</td>
    <td>Char</td>  
    <td>Text description if probability is outside table range.</td>
  </tr>
  <tr>
    <td>KlmCallP_0.05</td>
    <td>Char</td>  
    <td>Kolmogorov-Smirnov result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>2.  Lilliefors test with group-by option.</p>

In [ ]:
call val.td_analyze('KSTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=L;
                          groupby=tot_cust_years;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, Lilliefors is the UPI.)

teststyle = l</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Name</th>
    <th>Type</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td>Lilliefors</td>
    <td>Float</td>  
    <td>Lilliefors statistic.</td>
  </tr>
  <tr>
    <td>M</td>
    <td>Float</td>  
    <td>Count.</td>
  </tr>
  <tr>
    <td>LillieforsPValue</td>
    <td>Float</td>  
    <td>Probability associated with Lilliefors statistic.</td>
  </tr>
  <tr>
    <td>LillieforsPText</td>
    <td>Char</td>  
    <td>Text description if probability is outside table range.</td>
  </tr>
  <tr>
    <td>LillieforsCallP_0.05</td>
    <td>Char</td>  
    <td>Lilliefors result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>3.  Shapiro-Wilk test with group-by option</p>

In [ ]:
call val.td_analyze('KSTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=SW;
                          groupby=tot_cust_years;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, Shw is the UPI.)

teststyle = sw</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Name</th>
    <th>Type</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td>Shw</td>
    <td>Float</td>  
    <td>Shapiro-Wilk statistic.</td>
  </tr>
  <tr>
    <td>N</td>
    <td>Float</td>  
    <td>Count.</td>
  </tr>
  <tr>
    <td>ShapiroWilkPValue</td>
    <td>Float</td>  
    <td>Probability associated with Shapiro-Wilk statistic.</td>
  </tr>
  <tr>
    <td>ShapiroWilkPText</td>
    <td>Char</td>  
    <td>Text description if probability is outside table range.</td>
  </tr>
  <tr>
    <td>ShapiroWilkCallP_0.05.05</td>
    <td>Char</td>  
    <td>Shapiro-Wilk result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>4.  D'Agostino and Pearson Test with group-by option.</p>

In [ ]:
call val.td_analyze('KSTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=P;
                          groupby=tot_cust_years;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>5.  Smirnov test with group-by option.</p>

In [ ]:
call val.td_analyze('KSTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          columns=female_ind;
                          teststyle=S;
                          groupby=tot_cust_years;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there is a groupby column, it comprises the UPI. Otherwise, M is the UPI.)

teststyle = s</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Name</th>
    <th>Type</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td>M</td>
    <td>Float</td>  
    <td>Number of first distribution observations.</td>
  </tr>
  <tr>
    <td>N</td>
    <td>Float</td>  
    <td>Number of second distribution observations.</td>
  </tr>
  <tr>
    <td>D</td>
    <td>Float</td>  
    <td>D statistic.</td>
  </tr>
  <tr>
    <td>SmirnovPValue</td>
    <td>Float</td>  
    <td>Probability associated with D statistic.</td>
  </tr>
  <tr>
    <td>SmirnovPText</td>
    <td>Char</td>  
    <td>Text description if probability is outside table range.</td>
  </tr>  
  <tr>
    <td>ShapiroWilkCallP_0.05.05</td>
    <td>Char</td>  
    <td>Shapiro-Wilk result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>6.  Kolmogorov-Smirnov test that stores its results in an output table val_ks_out1 and does not contain a group-by option.

In [ ]:
call val.td_analyze('KSTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          teststyle=KS; 
                          statsdatabase=val;
                          outputdatabase=demo_user;
                          outputtablename=_val_ks_out1');

In [ ]:
SELECT * FROM demo_user._val_ks_out1;

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Parametric Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Parametric tests make assumptions about the data, such as the observations being normally distributed.  This can be verified with a test of normality prior to executing a parametric test.  Both T-Tests and F-Tests are provided.  T-Tests may be either paired or unpaired, while the unpaired T-Tests may be with or without an indicator variable.</p>  

<p style = 'font-size:16px;font-family:Arial'>F-Tests may be 1-way, 2-way or 3-way.  2-way tests may have equal or unequal cell counts (distinct column values), while the 3-way test must have equal cell counts.  A 1-way test has 1 independent input column, a 2-way test has 2 independent columns and a 3-way test has 3 independent columns in addition to a dependent “column of interest”.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **parametrictest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The parametrictest parameter:
    <li>Is required</li>
    <li>Must be the first parameter</li>
    <li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The column representing the dependent variable in an F-Test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The columns representing independent variables to be analyzed in a F-Test N-Way with Equal Cell Counts analysis.  There may be 1, 2 or 3 columns listed in this parameter.  If 2 or 3 columns, the number of distinct values for each combination of column values should be the same.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **equalvariance**</b></p>

<p style = 'font-size:16px;font-family:Arial'>An option available with the T-Test where, if true, the variance of the two samples (columns) is assumed to be equal.  The default assumption is that the variances are not equal.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>For a T-Test, the column representing the first variable to analyze. It must reside in the table indicated by the tablename parameter.  For an F-Test, the column representing the first independent variable in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumnvalues**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is a required option for a 2-way F-Test with Unequal Cell Counts.  It is a list of the firstcolumn values to be included in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>For a T-Test analysis, when gensqlonly is true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This option is not available for an F-Test analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **paired**</b></p>

<p style = 'font-size:16px;font-family:Arial'>An option of the T-Test, which when set to true, indicates that the first and second column values are matched with each other and the mean difference is analyzed.  The paired option defaults to false.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>For a T-Test, the column representing the second variable to analyze. It must reside in the table indicated by the tablename parameter.  If the withindicator option is set to true, the second column is used to define two analysis categories, one where the second column is negative or zero, and another where the second column is positive.  For an F-Test, the column representing the second independent variable in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumnvalues**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is a required option for a 2-way F-Test with Unequal Cell Counts.  It is a list of the secondcolumn values to be included in the analysis.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are t, fnway and f2way, as outlined below. The default is t for T-Test.
    
<li>T-Test paired, unpaired or unpaired with indicator variable (second column)</li>
<li>F-Test N-Way with Equal Cell Counts (1, 2 or 3 columns with same number of distinct values)</li>
<li>F-Test 2-Way with Unequal Cell Counts (2 columns with possibly different numbers of distinct values)</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this  "alpha" probability the null hypothesis is rejected.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **withindicator**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This option may be used with an un-paired T-Test, that is when teststyle=t and paired=false.  If the withindicator option is set to true, the second column is used to indicate there are two analysis categories, one for the case where the second column is negative or zero, and another when the second column is positive.</p>

--------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  T-Test producing an unpersisted result set with default values for teststyle (t) and  thresholdprobabilty (.05). </p> 

In [ ]:
call val.td_analyze('ParametricTest', 
                         'database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumn=cc_avg_bal;
                          secondcolumn=sv_avg_bal;
                          paired=true;
                          equalvariance=true;
                          groupby=tot_age,female_ind;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(Any groupby columns comprise the Unique Primary Index (UPI).)

teststyle = t(Default)</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td>D_F</td>
    <td>Float</td>  
    <td>Degrees of freedom for groupby columns.</td>
  </tr>
  <tr>
    <td>T</td>
    <td>Float</td>  
    <td>T statistic.</td>
  </tr>
  <tr>
    <td>TTestPValue</td>
    <td>Float</td>  
    <td>Probability associated with T statistic.</td>
  </tr>
  <tr>
    <td>TTestCallP.05.05</td>
    <td>Char</td>  
    <td>T-test result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis(positive)</li>
        <li>n=reject null hypothesis (negative)</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>2.  One-Way F-Test producing an unpersisted result set.</p>

In [ ]:
call val.td_analyze('ParametricTest',
                         'teststyle=fnway;
                          database=DEMO_Financial;
                          tablename=Customer;
                          columnofinterest=income;
                          columns=years_with_bank;
                          groupby=gender;
                          thresholdprobability=0.01;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, DF is the UPI.)

teststyle = fnway</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Name</th>
    <th>Type</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td>DF</td>
    <td>Integer</td>  
    <td>Degrees of freedom for variable.</td>
  </tr>
  <tr>
    <td>DFErr</td>
    <td>Integer</td>  
    <td>Degrees of freedom for error.</td>
  </tr>
  <tr>
    <td>F</td>
    <td>Float</td>  
    <td>F statistic.</td>
  </tr>
  <tr>
    <td>FPValue</td>
    <td>Float</td>  
    <td>Probability associated with F statistic.</td>
  </tr> 
  <tr>
    <td>FPText</td>
    <td>Char</td>  
    <td>If not NULL, the output appears in text format in the column.</td>
  </tr>   
  <tr>
    <td>FCallP</td>
    <td>Char</td>  
    <td>F-test result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis(positive)</li>
        <li>n=reject null hypothesis (negative)</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>3.  2-way F-Test with Unequal Cell Counts producing an unpersisted result set.</p>

In [ ]:
call val.td_analyze('ParametricTest',
                         'teststyle=f2way;
                          database=DEMO_Financial;
                          tablename=Customer;
                          columnofinterest=income;
                          firstcolumn=years_with_bank;
                          secondcolumn=gender;
                          thresholdprobability=0.05;
                          firstcolumnvalues=0,1,2,3,4,5,6,7;
                          secondcolumnvalues=M,F;
                          statsdatabase=val;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, DF is the UPI.)

teststyle = 2way</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Name</th>
    <th>Type</th>
    <th>Definition</th>
  </tr>
  <tr>
    <td>DF</td>
    <td>Integer</td>  
    <td>Degrees of freedom for model.</td>
  </tr>
  <tr>
    <td>Fmodel</td>
    <td>Float</td>  
    <td>F statistic for model.</td>
  </tr>  
  <tr>
    <td>DFErr</td>
    <td>Integer</td>  
    <td>Degrees of freedom for error.</td>
  </tr>
  <tr>
    <td>DF_1</td>
    <td>Integer</td>  
    <td>Degrees of freedom for first variable.</td>
  </tr>
  <tr>
    <td>F1</td>
    <td>Float</td>  
    <td>F statistic for first variable.</td>
  </tr> 
  <tr>
    <td>DF_2</td>
    <td>Integer</td>  
    <td>Degrees of freedom for second variable.</td>
  </tr>   
  <tr>
    <td>F2</td>
    <td>Float</td>  
    <td>F statistic for second variable.</td>
  </tr>    
</table>
</p>   
<p style = 'font-size:16px;font-family:Arial'>The description for other output columns can be found at the link <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_Lake_VMware/Vantage-Analytics-Library-User-Guide/Statistical-Tests/Parametric-Tests/Results-Data'>here.</a></p>


---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. Rank Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Statistical tests of this type calculate statistics based on the rank of variables rather than variable values.  In general, data that are ranked and ordinal may be analyzed by these tests.  Within some restraints, either numeric or non-numeric data may be analyzed.  Supported rank tests include the following:

<li>Mann-Whitney/Kruskal-Wallis Test</li>
<li>Wilcoxon Signed Ranks Test</li>
<li>Friedman Test with Kendall’s Coefficient of Concordance & Spearmans’ Rho</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'>The choice between the Mann-Whitney and Kruskal-Wallis tests is made automatically, looking at the number of distinct values of the independent variable.  A variation of the Mann-Whitney test considers each requested variable individually, rather than combined, performing a series of independent tests.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **ranktest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The ranktest parameter:
<li>Is required</li>
<li>Must be the first parameter</li>
<li>Is always enclosed in single quotes</li>
</p>    

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **blockcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Friedman test, the column that represents blocks. They must reside in the table indicated by the tablename parameter.  Note that when pairing treatment and block column values, a division by zero error may occur if unequal cell counts are found.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney and Friedman tests, the column specified with this parameter indicates the dependent variable. If non-numeric, it will be ranked alphanumerically.  It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney test, the columns specified with this parameter represent the independent variables. They must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Wilcoxon test, the column that represents the first sample variable. It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.  This parameter is not allowed when using the Mann-Whitney test.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>One or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **includezero**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Wilcoxon test.  Ordinarily, the Wilcoxon test discards cases with zero differences.  This option, when set to true, includes these cases with the positive count.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **independent**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney test, its selection by setting independent=true, indicates that a variation of the Mann-Whitney test should be performed, considering each requested variable individually, rather than in combination, performing a series of independent tests.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Wilcoxon test, the column that represents the second sample variable. It must reside in the table indicated by the tablename parameter.  Note that when pairing treatment and block column values, a division by zero error may occur if unequal cell counts are found.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **singletail**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Mann-Whitney and Wilcoxon tests.  If the Mann-Whitney test becomes a Kruskall-Wallis test, the singletail option is invalid.  By default, a two-tailed test is performed.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are mw, friedman and wilcoxon, representing a Mann-Whitney, Friedman or Wilcoxon test.  If not specified, the default test style is Mann-Whitney (mw).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this "alpha" probability the null hypothesis is rejected.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **treatmentcolumn**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Friedman test, the column that represents the independent categorical variable. They must reside in the table indicated by the tablename parameter.</p>

--------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Mann-Whitney test with a threshold probability of 0.01.</p>

In [ ]:
call val.td_analyze('ranktest',
                         'teststyle=mw;
                          statsdatabase=val;
                          database=DEMO_Financial;
                          tablename=Customer;
                          columnofinterest=income;
                          columns=gender;
                          groupby=years_with_bank;
                          thresholdprobability=0.01;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, Z is the UPI.)

teststyle = mw (Default)</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Z</td>
    <td>Float</td>  
    <td>Mann-Whitney Z statistic.</td>
  </tr>
  <tr>
    <td>MannWhitneyPValue</td>
    <td>Float</td>  
    <td>Probability associated with Mann-Whitney statistic.</td>
  </tr>
  <tr>
    <td>MannWhitneyCallP_0.01</td>
    <td>Char</td>  
    <td>Mann-Whitney result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>2.  A set of Mann-Whitney independent tests.  The threshold probability assumes the default value of 0.05.</p>

In [ ]:
call val.td_analyze('ranktest',
                         'teststyle=mw;
                          statsdatabase=val;
                          independent=true;
                          database=demo_user;
                          tablename=VAL_ADS;
                          columnofinterest=tot_income;
                          columns=female_ind,cc_acct_ind,ck_acct_ind,sv_acct_ind; ');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, Z is the UPI.)
The output is similar in the step above but the threshold probability is changed from 0.01 to 0.05</p>


<p style = 'font-size:16px;font-family:Arial'>3.  Wilcoxon Test - the threshold probability assumes the default value of 0.05.</p>

In [ ]:
call val.td_analyze('ranktest',
                         'teststyle=wilcoxon;
                          statsdatabase=val;
                          database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumn=ck_avg_bal;
                          secondcolumn=sv_avg_bal;
                          groupby=tot_cust_years; ');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, Z_ is the UPI.)

teststyle = wilcoxon</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>N</td>
    <td>Integer</td>  
    <td>Variable count.</td>
  </tr>
  <tr>
    <td>Z_</td>
    <td>Float</td>  
    <td>Mann-Whitney Z statistic.</td>
  </tr>
  <tr>
    <td>WilcoxonPValue</td>
    <td>Float</td>  
    <td>Probability associated with Wilcoxon statistic.</td>
  </tr>  
  <tr>
    <td>WilcoxonCallP_0.05</td>
    <td>Char</td>  
    <td>Wilcoxon result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis(positive)</li>
        <li>n=reject null hypothesis(negative)</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>4.  A Friedman Test using a specially prepared input table called VAL_Friedman_WorkTable.  To build the VALFriedmanWorkTable, the following SQL may be used.  Note that the value ‘18’ is the smallest count of value combinations in the gender and marital_status columns in the Customer table.  The value '18'  is determined using the following query:</p>

In [ ]:
SELECT 
     MIN("T0"."_val_N") AS "smallest_count"
FROM 
( SELECT 
     "S0"."marital_status" AS "marital_status"
    ,"S0"."gender" AS "gender"
    ,COUNT(*) AS "_val_N"
FROM "DEMO_Financial"."Customer" AS "S0"
GROUP BY "S0"."marital_status", "S0"."gender"
) AS "T0"
;

In [ ]:
CREATE SET TABLE "demo_user"."VAL_Friedman_Worktable" AS (
SELECT
    "cust_id"
    ,"gender"
    ,"marital_status"
    ,"income"
    ,"ckacct"
    ,"svacct"
    ,SAMPLEID AS "xSampleId"
FROM "DEMO_Financial"."Customer_Analysis"
SAMPLE 
    WHEN gender='f' and marital_status='1' THEN 18
    WHEN gender='f' and marital_status='2' THEN 18
    WHEN gender='f' and marital_status='3' THEN 18
    WHEN gender='f' and marital_status='4' THEN 18
    WHEN gender='m' and marital_status='1' THEN 18
    WHEN gender='m' and marital_status='2' THEN 18
    WHEN gender='m' and marital_status='3' THEN 18
    WHEN gender='m' and marital_status='4' THEN 18
END
) WITH DATA PRIMARY INDEX("cust_id");

In [ ]:
call val.td_analyze('ranktest',
                         'teststyle=friedman;
                          statsdatabase=val;
                          database=demo_user;
                          tablename=VAL_Friedman_Worktable;
                          columnofinterest=income;
                          treatmentcolumn=gender;
                          blockcolumn=marital_status;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, Kendalls_W is the UPI.)

teststyle = friedman</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Kendalls_W</td>
    <td>Float</td>  
    <td>Kendall's W statistic.</td>
  </tr>
  <tr>
    <td>Average_Spearmans_Rho</td>
    <td>Float</td>  
    <td>Average Spearman's Rho.</td>
  </tr>
  <tr>
    <td>DF_1</td>
    <td>Integer</td>  
    <td>Degrees of freedom for treatments.</td>
  </tr> 
  <tr>
    <td>DF_2</td>
    <td>Integer</td>  
    <td>Degrees of freedom for blocks.</td>
  </tr> 
  <tr>
    <td>F</td>
    <td>Float</td>  
    <td>2-way ANOVA F statistic on ranks.</td>
  </tr>
  <tr>
    <td>FriedmanPValue</td>
    <td>Float</td>  
    <td>Probability associated with Friedman statistic.</td>
  </tr>
  <tr>
    <td>FriedmanPText</td>
    <td>Char</td>  
    <td>Text description of probability if out of table range.</td>
  </tr>
  <tr>
    <td>FriedmanCallP_0.05</td>
    <td>Char</td>  
    <td>Friedman result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis(positive)</li>
        <li>n=reject null hypothesis(negative)</li>
    </td>
  </tr>    
</table>
   
<p>

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. Chi Square Tests</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Purpose</b></p>

<p style = 'font-size:16px;font-family:Arial'>Statistical tests of this type are based on a matrix of frequencies or counts.  A frequency pattern that is non-random is sought in the matrix.  Supported tests of this type include the following:

<li>Chi Square Test</li>
</p>

<p style = 'font-size:16px;font-family:Arial'>Besides a Chi Square value, other measures are computed in a Chi Square Test, including a Phi Coefficient,  Cramer’s V, Likelihood Ratio Chi Square, Continuity-Adjusted Chi Square and Contingency Coefficient.

<li>Median Test</li>
</p>
    
<p style = 'font-size:16px;font-family:Arial'>A Median Test is a variation of Chi Square Test wherein samples are tested to see if their populations have the same median value.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Required Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **chisquaretest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The chisquaretest parameter:
    
<li>Is required</li>
<li>Must be the first parameter</li>
<li>Is always enclosed in single quotes</li>
</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **database**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The database containing the table to analyze.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **tablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The table containing the columns to analyze.  It must reside in the database indicated by the database parameter.</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Optional Parameters</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columnofinterest**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Median Test, the column specified with this parameter indicates the dependent variable (numeric).  It must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **columns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Median Test, the columns specified with this parameter represent the independent variables (categorical). They must reside in the table indicated by the tablename parameter.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **fallback**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, FALLBACK is requested as an attribute of the created output results table.  When false or parameter not specified, the FALLBACK attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **firstcolumns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Chi Square Test, one or more columns representing the first of variable pairs for analysis. They must reside in the table indicated by the tablename parameter.  Note that the number of combinations of firstcolumns and secondcolumns may not exceed 100.  Note also that if the product of the number distinct values in these column pairs exceeds 2000, the analysis of that combination is skipped (but not if gensqlonly is true).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **gensqlonly**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, the SQL for the requested function is returned as a result set but not run. When not specified or set to false, the SQL is run but not returned.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **groupby**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Available only in the Median Test, one or more group-by columns may optionally be specified so that a separate result is produced for each value or combination of values in the specified column or columns.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **multiset**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When true, MULTISET is requested as an attribute of the created output results table.  When false or parameter not specified, the MULTISET attribute is not included in the create table SQL.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the database to contain the analysis results table.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **outputtablename**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Specifies the name of the table to store the analysis results. If not supplied, the results are returned as a result set.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **overwrite**</b></p>

<p style = 'font-size:16px;font-family:Arial'>When overwrite is set to true or not set, the output table is dropped before creating a new one.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **secondcolumns**</b></p>

<p style = 'font-size:16px;font-family:Arial'>Used only by the Chi Square Test, one or more columns representing the second of variable pairs for analysis. They must reside in the table indicated by the tablename parameter.  Note that the number of combinations of firstcolumns and secondcolumns may not exceed 100.  Note also that if the product of the number distinct values in these column pairs exceeds 2000, the analysis of that combination is skipped (but not if gensqlonly is true).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **statsdatabase**</b></p>

<p style = 'font-size:16px;font-family:Arial'>This is the database where the statistical test metadata tables are installed.  If not specified, the source database is searched for these metadata tables.</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **teststyle**</b></p>

<p style = 'font-size:16px;font-family:Arial'>The valid values are chisq and median, representing a Chi Square or Median test.  If not specified, the default test style is Chi Square (chisq).</p>

<p style = 'font-size:16px;font-family:Arial'><b>- **thresholdprobability**</b></p>

<p style = 'font-size:16px;font-family:Arial'>You may optionally change the threshold probability or allow it to default to 0.05.  Below this "alpha" probability the null hypothesis is rejected.</p>

---------------------------

<p style = 'font-size:16px;font-family:Arial'>1.  Chi Square Test with non-persisted results.</p>

In [ ]:
call val.td_analyze('chisquaretest',
                         'statsdatabase=val;
                          teststyle=chisq;
                          database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumns=female_ind,single_ind;
                          secondcolumns=sv_acct_ind,cc_acct_ind,ck_acct_ind;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(Column1 is the UPI.)

teststyle = chisq (Default)</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>column1</td>
    <td>Char</td>  
    <td>First of pair of variables.</td>
  </tr>
  <tr>
    <td>column2</td>
    <td>Char</td>  
    <td>Second of pair of variables.</td>
  </tr>
  <tr>
    <td>Chisq</td>
    <td>Float</td>  
    <td>Chi-Squared statistic.</td>
  </tr>  
  <tr>
    <td>DF</td>
    <td>Integer</td>  
    <td>Degrees of freedom.</td>
  </tr>  
  <tr>
    <td>Z</td>
    <td>Float</td>  
    <td>Z-score</td>
  </tr>  
  <tr>
    <td>CramersV</td>
    <td>Float</td>  
    <td>Cramer'sV.</td>
  </tr>  
  <tr>
    <td>PhiCoeff</td>
    <td>Float</td>  
    <td>Phi coefficient.</td>
  </tr>  
  <tr>
    <td>LlhChiSq</td>
    <td>Float</td>  
    <td>Likelihood ratio Chi-Squared statistic.</td>
  </tr>  
  <tr>
    <td>ContAdjChiSq</td>
    <td>Float</td>  
    <td>Continuity-adjusted Chi-Squared statistic.</td>
  </tr>  
  <tr>
    <td>ContinCoeff</td>
    <td>Float</td>  
    <td>Contingency coefficient.</td>
  </tr>  
  <tr>
    <td>ChiPValue</td>
    <td>Float</td>  
    <td>Probability associated with Chi-Squared statistic.</td>
  </tr>  
  <tr>
    <td>ChiPText</td>
    <td>Float</td>  
    <td>Text description if probability is outside table range.</td>
  </tr>  
  <tr>
    <td>ChiCallP_0.05</td>
    <td>Char</td>  
    <td>Chi-Squared result:
        <li>a=accept null hypothesis (not significantly different)</li>
        <li>p=reject null hypothesis(positive) (significantly different)</li>
        <li>n=reject null hypothesis(negative)</li>
    </td>
  </tr>    
</table>
   
<p>

<p style = 'font-size:16px;font-family:Arial'>2.  Chi Square Test with results persisted in an output table.</p>

In [ ]:
call val.td_analyze('chisquaretest',
                         'statsdatabase=val;
                          teststyle=chisq;
                          database=demo_user;
                          tablename=VAL_ADS;
                          firstcolumns=female_ind,single_ind;
                          secondcolumns=sv_acct_ind,cc_acct_ind,ck_acct_ind;
                          outputdatabase=demo_user;
                          outputtablename=_val_chisquare_out;');

In [ ]:
SELECT * FROM demo_user._val_chisquare_out ORDER BY 1;

<p style = 'font-size:16px;font-family:Arial'>3.  Median Test with groupby column years_with_bank, thresholdprobability set to 0.01 and results returned in a result set.</p> 

In [ ]:
call val.td_analyze('chisquaretest', 
                         'statsdatabase=val;
                          teststyle=median;
                          database=DEMO_Financial;
                          tablename=Customer;
                          columnofinterest=income;
                          columns=marital_status;
                          groupby=years_with_bank;
                          thresholdprobability=0.01;');

<p style = 'font-size:16px;font-family:Arial'>The above output has the output columns as below(If there are groupby columns, they comprise the UPI. Otherwise, ChiSq is the UPI.)

teststyle = median</p>

 <table style = 'font-size:14px;font-family:Arial'>
  <tr>
    <th>Column</th>
    <th>DataType</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Chisq</td>
    <td>Float</td>  
    <td>Chi-Squared statistic.</td>
  </tr>  
  <tr>
    <td>DF</td>
    <td>Integer</td>  
    <td>Degrees of freedom.</td>
  </tr>  
  <tr>
    <td>MedianPValue</td>
    <td>Float</td>  
    <td>Probability associated with Chi-Squared statistic.</td>
  </tr>  
  <tr>
    <td>MedianPText</td>
    <td>Char</td>  
    <td>Text description if probability is outside table range.</td>
  </tr>  
  <tr>
    <td>MedianCallP_0.01</td>
    <td>Char</td>  
    <td>Chi-Squared result:
        <li>a=accept null hypothesis</li>
        <li>p=reject null hypothesis(positive)</li>
        <li>n=reject null hypothesis(negative)</li>
    </td>
  </tr>    
</table>
   
<p>

---

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>15. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:18px;font-family:Arial'>Clean up work tables to prevent error next time.</p>

In [ ]:
DROP TABLE "demo_user"."VAL_Friedman_Worktable";

In [ ]:
DROP TABLE "demo_user"."VAL_ADS";

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:18px;font-family:Arial'>The following code will clean up tables and databases created above.</p>
<p style = 'font-size:16px;font-family:Arial'><b>Note:</b> In case you plan to execute the other notebooks in the VantageAnalyticLibrary , do not run the <b>remove_data</b> procedure below. Also in case you are not removing the databases and tables here , please make sure that you do not execute the <b>get_data</b> procedure in the next VantageAnalyticLibrary notebook.</p>

In [ ]:
Database demo_user;

In [ ]:
call remove_data('DEMO_Financial');

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>